In [1]:
import duckdb
import pandas as pd

In [8]:
path_to_read = "../data/raw"
path_to_save = "../data/processed"

In [7]:
data_com = duckdb.read_csv(f"{path_to_read}/communes-france-2025.csv")
data_com

┌──────────┬────────────┬─────────────────────────┬───────────────────────┬─────────────────────────┬────────────────────────────┬─────────────────────────┬─────────────────────────┬─────────┬───────────────┬──────────┬──────────────────────┬──────────┬──────────────┬─────────────┬────────────────────────────┬───────────┬───────────────────────────────────────────┬───────────────┬───────────────┬─────────────┬─────────────────────────────────────────────────┬─────────────┬───────────────────────────────┬────────────────────┬────────────────────┬──────────────────────┬────────────────────────────┬──────────────────────────────┬────────────┬────────────────────┬────────────────┬─────────┬──────────────────┬───────────────────┬───────────────────┬─────────────────┬──────────────────┬─────────────────┬──────────────────┬────────────────┬────────────────────────────────┬─────────────────────────────┬─────────────────────────────────────────────────────┬──────────────────────────────────────

In [10]:
data_com_processed = duckdb.sql("""
SELECT 
           code_insee,
           nom_standard,
           nom_sans_accent,
           reg_code,
           reg_nom,
           dep_code,
           dep_nom,
           epci_code,
              epci_nom,
           population,
          superficie_hectare,
           superficie_km2,
           densite AS densite_hab_km2,

           
FROM data_com 

           """)

data_com_processed.to_csv(f"{path_to_save}/communes_france_2025.csv")

In [23]:
query = """ 
SELECT * 
FROM data_com_processed
WHERE (epci_code NOT LIKE '2%') OR (epci_code IS NULL)
"""

duckdb.sql(query)

┌────────────┬───────────────────┬───────────────────┬──────────┬──────────────────┬──────────┬───────────────┬───────────┬────────────┬────────────┬────────────────────┬────────────────┬─────────────────┐
│ code_insee │   nom_standard    │  nom_sans_accent  │ reg_code │     reg_nom      │ dep_code │    dep_nom    │ epci_code │  epci_nom  │ population │ superficie_hectare │ superficie_km2 │ densite_hab_km2 │
│  varchar   │      varchar      │      varchar      │  int64   │     varchar      │ varchar  │    varchar    │  varchar  │  varchar   │   int64    │       int64        │     int64      │     double      │
├────────────┼───────────────────┼───────────────────┼──────────┼──────────────────┼──────────┼───────────────┼───────────┼────────────┼────────────┼────────────────────┼────────────────┼─────────────────┤
│ 22016      │ Île-de-Bréhat     │ ile-de-brehat     │       53 │ Bretagne         │ 22       │ Côtes-d'Armor │ ZZZZZZZZZ │ Sans objet │        402 │                347 │      

In [47]:

query = """
SELECT 
           epci_code AS siren,
           epci_nom,
           count(*) AS n_communes,
           count(DISTINCT dep_code) AS n_departements,
           count(DISTINCT reg_code) AS n_regions,
           sum(population) AS population,
           sum(superficie_hectare) AS superficie_hectare,
           sum(superficie_km2) AS superficie_km2,
           round(sum(population) / sum(superficie_km2),2) AS densite_hab_km2
FROM data_com_processed
WHERE (epci_code IS NOT NULL) AND ((epci_code LIKE '2%'))
GROUP BY epci_code,epci_nom"""

data_epci_processed = duckdb.sql(query)

In [48]:
data_epci_processed

┌───────────┬─────────────────────────────────────────┬────────────┬────────────────┬───────────┬────────────┬────────────────────┬────────────────┬─────────────────┐
│   siren   │                epci_nom                 │ n_communes │ n_departements │ n_regions │ population │ superficie_hectare │ superficie_km2 │ densite_hab_km2 │
│  varchar  │                 varchar                 │   int64    │     int64      │   int64   │   int128   │       int128       │     int128     │     double      │
├───────────┼─────────────────────────────────────────┼────────────┼────────────────┼───────────┼────────────┼────────────────────┼────────────────┼─────────────────┤
│ 248500415 │ CC du Pays de la Châtaigneraie          │         14 │              1 │         1 │      14856 │              25830 │            257 │           57.81 │
│ 200072882 │ CC de Vie et Boulogne                   │         15 │              1 │         1 │      45851 │              46868 │            468 │           97.97 

In [30]:
data_epci = duckdb.read_csv(f"{path_to_read}/perimetre-epci-a-fp.csv",ignore_errors=True)
data_epci

┌─────────┬───────────┬────────────────────────────────────┬──────────────────┬─────────────┬────────────┬───────────────┬───────────────┬─────────┬─────────┬──────────────┬────────────────────────┬───────────┬───────────┐
│  dept   │   siren   │           raison_sociale           │ nature_juridique │ mode_financ │ nb_membres │ total_pop_tot │ total_pop_mun │ dep_com │  insee  │ siren_membre │       nom_membre       │ ptot_2025 │ pmun_2025 │
│ varchar │   int64   │              varchar               │     varchar      │   varchar   │   int64    │    varchar    │    varchar    │ varchar │ varchar │    int64     │        varchar         │  varchar  │  varchar  │
├─────────┼───────────┼────────────────────────────────────┼──────────────────┼─────────────┼────────────┼───────────────┼───────────────┼─────────┼─────────┼──────────────┼────────────────────────┼───────────┼───────────┤
│ 01      │ 200029999 │ CC Rives de l'Ain - Pays du Cerdon │ CC               │ FPU         │         14 │ 1

In [44]:
test = duckdb.sql("select count(*),     siren from data_epci WHERE siren is NULL group by siren")
test

┌──────────────┬───────┐
│ count_star() │ siren │
│    int64     │ int64 │
├──────────────┼───────┤
│            1 │  NULL │
└──────────────┴───────┘

In [43]:
query = """ 
SELECT test.*,
data_epci_processed.*
FROM test
FULL JOIN data_epci_processed
ON test.siren = data_epci_processed.epci_code
WHERE data_epci_processed.epci_code IS NULL OR test.siren IS NULL
"""

duckdb.sql(query)

┌──────────────┬───────┬───────────┬──────────┬────────────┬────────────────┬───────────┬────────────┬────────────────────┬────────────────┬─────────────────┐
│ count_star() │ siren │ epci_code │ epci_nom │ n_communes │ n_departements │ n_regions │ population │ superficie_hectare │ superficie_km2 │ densite_hab_km2 │
│    int64     │ int64 │  varchar  │ varchar  │   int64    │     int64      │   int64   │   int128   │       int128       │     int128     │     double      │
├──────────────┼───────┼───────────┼──────────┼────────────┼────────────────┼───────────┼────────────┼────────────────────┼────────────────┼─────────────────┤
│            1 │  NULL │ NULL      │ NULL     │       NULL │           NULL │      NULL │       NULL │               NULL │           NULL │            NULL │
└──────────────┴───────┴───────────┴──────────┴────────────┴────────────────┴───────────┴────────────┴────────────────────┴────────────────┴─────────────────┘

In [84]:
query = """ 
SELECT 
    ANY_VALUE(e2.dept) AS dept,
    e2.siren AS siren,
    ANY_VALUE(e2.total_pop_tot) AS total_pop_tot,
    ANY_VALUE(e2.total_pop_mun) AS total_pop_mun,
    ANY_VALUE(e1.n_communes) AS n_communes,
    ANY_VALUE(e1.n_departements) AS n_departements,
    ANY_VALUE(e1.n_regions) AS n_regions,
    ANY_VALUE(e1.superficie_hectare) AS sperficie_hectare,
    ANY_VALUE(e1.superficie_km2) AS superficie_km2,
    ANY_VALUE(e1.densite_hab_km2) AS densite_hab_km2
FROM data_epci AS e2
FULL JOIN data_epci_processed AS e1
ON e1.siren = e2.siren
WHERE e2.siren IS NOT NULL 
GROUP BY 
    e2.siren
ORDER BY dept
"""

data_epci_processed= duckdb.sql(query)

In [67]:
tableur_epci = pd.read_csv(f"{path_to_read}/tableur_epci.csv",sep=",")
tableur_epci

,Nom du groupement,Nature juridique,N° SIREN,Département,Population totale,Population DGF,Densité démographique (hab/km²),Superficie,Nombre de membres,Nombre total des délégués,Nombre de compétences exercées,Potentiel fiscal,Dotation globale,Dotation de compensation,Dotation d'intercommunalité,Ville du siège,Nom membre,Siren membre,Population totale du membre du groupement
0,CC Rives de l'Ain - Pays du Cerdon,CC,200029999,01 - Ain,15156,15806.0,"89,2",NaN,14,37,21,5493274.0,844272.0,527981.0,316291.0,Jujurieux,Poncin,210103032.0,1792.0
1,CC Rives de l'Ain - Pays du Cerdon,CC,200029999,01 - Ain,15156,15806.0,"89,2",NaN,14,37,21,5493274.0,844272.0,527981.0,316291.0,Jujurieux,Neuville-sur-Ain,210102737.0,1823.0
2,CC Rives de l'Ain - Pays du Cerdon,CC,200029999,01 - Ain,15156,15806.0,"89,2",NaN,14,37,21,5493274.0,844272.0,527981.0,316291.0,Jujurieux,Jujurieux,210101994.0,2246.0
3,CC Rives de l'Ain - Pays du Cerdon,CC,200029999,01 - Ain,15156,15806.0,"89,2",NaN,14,37,21,5493274.0,844272.0,527981.0,316291.0,Jujurieux,Saint-Alban,210103313.0,196.0
4,CC Rives de l'Ain - Pays du Cerdon,CC,200029999,01 - Ain,15156,15806.0,"89,2",NaN,14,37,21,5493274.0,844272.0,527981.0,316291.0,Jujurieux,Priay,210103149.0,1826.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35322,CA du Grand Nord de Mayotte,CA,200060465,976 - Mayotte,60372,70737.0,"693,9",NaN,4,40,31,4517268.0,3428523.0,NaN,3428523.0,Bandraboua,Mtsamboro,200008845.0,8025.0
35323,CC du Sud,CC,200060473,976 - Mayotte,31945,37555.0,"319,5",NaN,4,34,23,2802700.0,1760573.0,NaN,1760573.0,Bandrélé,Bandrele,200008738.0,10529.0
35324,CC du Sud,CC,200060473,976 - Mayotte,31945,37555.0,"319,5",NaN,4,34,23,2802700.0,1760573.0,NaN,1760573.0,Bandrélé,Bouéni,200008746.0,6503.0
35325,CC du Sud,CC,200060473,976 - Mayotte,31945,37555.0,"319,5",NaN,4,34,23,2802700.0,1760573.0,NaN,1760573.0,Bandrélé,Chirongui,200008779.0,9197.0


In [70]:
tableur_epci.columns

Index(['Nom du groupement', 'Nature juridique', 'N° SIREN', 'Département',
       'Population totale', 'Population DGF',
       'Densité démographique (hab/km²)', 'Superficie', 'Nombre de membres',
       'Nombre total des délégués', 'Nombre de compétences exercées',
       'Potentiel fiscal', 'Dotation globale', 'Dotation de compensation',
       'Dotation d'intercommunalité', 'Ville du siège', 'Nom membre',
       'Siren membre', 'Population totale du membre du groupement'],
      dtype='object')

In [71]:
mapping = {'Nom du groupement': 'epci_nom',
            'Nature juridique' : "nature_juridique",
              'N° SIREN':'siren', 
              'Département': 'dept',
       'Population totale': 'total_pop_mun',
         'Population DGF': 'total_pop_tot',
       'Densité démographique (hab/km²)': 'densite_par_km2' ,
         'Superficie': 'superficie_km2',
           'Nombre de membres': 'nb_membres',
       'Nombre total des délégués': 'nb_delegues', 
       'Nombre de compétences exercées': 'nb_competences',
       'Potentiel fiscal' : 'potentiel_fiscal', 'Dotation globale': 'dotation_globale', 'Dotation de compensation': 'dotation_compensation',
       "Dotation d'intercommunalité": 'dotation_intercommunalite', 'Ville du siège': 'ville_siege', 'Nom membre': 'nom_membre',
       'Siren membre': 'siren_membre', 'Population totale du membre du groupement': 'pop_totale_membre',
}

tableau_epci = tableur_epci.rename(columns=mapping)

In [77]:
tableau_epci_filtered = tableau_epci.drop(columns=[
    'dept',
    'superficie_km2',
    'nom_membre',
    'siren_membre',
    'pop_totale_membre'
    ])

In [86]:
query = """
SELECT
    t2.dept,
    t1.*,
    t2.n_departements AS nb_departements,
    t2.n_regions AS nb_regions,
    t2.sperficie_hectare AS superficie_hectare,
    t2.superficie_km2 AS superficie_km2
FROM tableau_epci_filtered AS t1
LEFT JOIN data_epci_processed AS t2
ON t1.siren = t2.siren
"""

tableau_final_epci = duckdb.sql(query)
tableau_final_epci

┌─────────┬────────────────────────────────────┬──────────────────┬───────────┬───────────────┬───────────────┬─────────────────┬────────────┬─────────────┬────────────────┬──────────────────┬──────────────────┬───────────────────────┬───────────────────────────┬─────────────────────┬─────────────────┬────────────┬────────────────────┬────────────────┐
│  dept   │              epci_nom              │ nature_juridique │   siren   │ total_pop_mun │ total_pop_tot │ densite_par_km2 │ nb_membres │ nb_delegues │ nb_competences │ potentiel_fiscal │ dotation_globale │ dotation_compensation │ dotation_intercommunalite │     ville_siege     │ nb_departements │ nb_regions │ superficie_hectare │ superficie_km2 │
│ varchar │              varchar               │     varchar      │   int64   │     int64     │    double     │     varchar     │   int64    │    int64    │     int64      │      double      │      double      │        double         │          double           │       varchar       │     

In [87]:
tableau_final_epci.write_csv(f"{path_to_save}/indicateurs_epci.csv")